
# Datové typy a objekty pro práci s časem


| Koncept          | Skalární Třída | Třída objektu / indexu      | Datový Typ v pandas | Primární metoda vytvoření  |
|----------------|----------------|-----------------|----------------------|----------------------------|
| Datum a čas  | Timestamp      | DatetimeIndex   | datetime64[ns] nebo datetime64[ns, tz] | to_datetime nebo date_range  |
| Časová delta   | Timedelta      | TimedeltaIndex  | timedelta64[ns]      | to_timedelta nebo timedelta_range |
| Časové rozpětí | Period         | PeriodIndex     | period[freq]         | Period nebo period_range   |
| Datumové posuny | DateOffset     | None            | None                 | DateOffset                 |

Zdroj: https://pandas.pydata.org/docs/user_guide/timeseries.html


- metoda `pd.to_datetime()` - převede vstupní hodnotu na typ `datetime64[ns]`, případně na typ `datetime64[ns, tz]` pokud je vstupní hodnota v časové zóně, viz https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html#pandas.Timestamp
- metoda `pd.to_timedelta()` - převede vstupní hodnotu na typ `timedelta64[ns]`, viz https://pandas.pydata.org/docs/reference/api/pandas.Timedelta.html#pandas.Timedelta
- metoda `pd.to_period()` - převede vstupní hodnotu na typ `period[freq]`, viz https://pandas.pydata.org/docs/reference/api/pandas.Period.html#pandas.Period

### Metoda to_datetime()
- Převede vstupní hodnotu na typ `datetime64[ns]`, případně na typ `datetime64[ns, tz]` pokud je vstupní hodnota v časové zóně.
- Očekává hodnotu pro rok jako 4 číselný řetězec.
- Ve výchozím nastavení předchází měsíc roku, lze změnit pomocí parametru `dayfirst`.
- V případě nestandardního formátu data lze použít parametr `format` a v něm definovat formát data, viz [https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior).
- Informace o čase je vrácena na základě systémových hodin počítače, lze změnit pomocí parametru `utc`.
- V případě potřeby změny časové zóny je možné použít metodu `tz_convert()` nebo `tz_localize()`, viz [https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html#pandas.Timestamp](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html#pandas.Timestamp). Informace o časových zónách (jejich názvech) poskytuje knihovna `pytz`, viz [https://pypi.org/project/pytz/](https://pypi.org/project/pytz/).


In [7]:
import pandas as pd  # Import knihovny pandas pro práci s daty

# Definice data z řetězce ve formátu 'yy-mm-dd'
datum = pd.to_datetime('22-01-15', format='%y-%m-%d')

# Definice data a času z řetězce ve standardním formátu 'yyyy-mm-dd hh:mm'
datum_cas = pd.to_datetime('2022-01-15 15:45')

# Definice data a času z aktuálního data s použitím UTC časové zóny
datum_cas_dnes = pd.to_datetime('today', utc=True)

# Definice data a času z aktuálního data s časem definovaným uživatelem,
# pokud je jako argument použit pouze čas, použije se dnešní datum
datum_cas_dnes2 = pd.to_datetime('15:45')

# Výpis definovaných dat a časů
print('datum cas:', datum_cas)
print('datum cas dnes:', datum_cas_dnes)
print('datum cas dnes2:', datum_cas_dnes2)


datum cas: 2022-01-15 15:45:00
datum cas dnes: 2024-03-24 16:25:14.606540+00:00
datum cas dnes2: 2024-03-24 15:45:00


In [8]:
import pandas as pd  # Import knihovny pandas pro práci s daty

# Vytvoření 'aware' Timestamp pro aktuální datum a čas v UTC
datum_cas_dnes_utc = pd.to_datetime('today', utc=True)

# Konverze tohoto Timestamp na časové pásmo Moskvy
datum_cas_dnes_moscow = datum_cas_dnes_utc.tz_convert('Europe/Moscow')

# Výpis obou Timestampů s příslušným časovým pásmem
print(f"UTC Timestamp: {datum_cas_dnes_utc}")  # Výpis času v UTC
print(f"Moscow Timestamp: {datum_cas_dnes_moscow}")  # Výpis převedeného času na časové pásmo Moskvy


UTC Timestamp: 2024-03-24 16:25:14.622836+00:00
Moscow Timestamp: 2024-03-24 19:25:14.622836+03:00


### Metoda to_timedelta()
- Převede vstupní hodnotu na typ `timedelta64[ns]`.
- Představuje rozdíl mezi dvěma časovými údaji / posun v čase.
- Rozdíl v čase lze pak snadno přičítat či odečítat od časového údaje (datetime).
- Možné hodnoty posunu jsou:
    - `ns` - nanosekundy
    - `us` - mikrosekundy
    - `ms` - milisekundy
    - `s` - sekundy
    - `m` - minuty
    - `h` - hodiny
    - `D` - dny
    - `W` - týdny

Kompletní přehled aliasů zde: [https://pandas.pydata.org/docs/reference/api/pandas.Timedelta.html#pandas.Timedelta](https://pandas.pydata.org/docs/reference/api/pandas.Timedelta.html#pandas.Timedelta)
 
Pozn.: Objekt typu timedelta vznikne automaticky při odečítání dvou objektů typu `datetime64`.


In [9]:
import pandas as pd  # Import knihovny pandas pro práci s daty

# Vytvoření 'naive' Timestamp pro 'dnes' s explicitním nastavením na UTC
datum_cas_dnes = pd.to_datetime('today', utc=True)

# Vytvoření Timedelta pro posun o 3 dny, 2 hodiny a 3 sekundy
posun = pd.to_timedelta('3 days 2 hours 3 seconds')

# Posunutí času o zadanou Timedelta
posunuto = datum_cas_dnes + posun

# Výpis aktuálního data a času a data a času po posunu
print('Datum a cas dnes:', datum_cas_dnes)
print('Posunuto:', posunuto)


Datum a cas dnes: 2024-03-24 16:25:14.625285+00:00
Posunuto: 2024-03-27 18:25:17.625285+00:00


### Metoda period()

- Jedná se o metodu objektu typu `datetime64[ns]`.
- Převede vstupní hodnotu na typ `period[freq]`, která nepředstavuje konkrétní bod v čase, ale rozsah času. Například `pd.to_datetime('today').to_period(freq='Y')` představuje celý rok dnešního dne, přičemž můžeme přistupovat k jednotlivým jeho částem (atributům). Více informací zde: [https://pandas.pydata.org/docs/reference/api/pandas.Period.html](https://pandas.pydata.org/docs/reference/api/pandas.Period.html).
- Přehled jednotlivých aliasů frekvence je k dispozici zde: [https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-period-aliases](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-period-aliases).
- V případě potřeby změny frekvence je možné použít metodu `asfreq()`. Více informací zde: [https://pandas.pydata.org/docs/reference/api/pandas.Period.html#pandas.Period](https://pandas.pydata.org/docs/reference/api/pandas.Period.html#pandas.Period).


In [10]:
import pandas as pd  # Import knihovny pandas pro práci s daty
import datetime  # Import modulu datetime pro práci s datem a časem

# Výpočet dnešního datumu a času pomocí datetime
dnes = datetime.datetime.now()
# Konverze dnešního datumu a času na pandas Period s denní frekvencí
dnes_perioda = pd.Period(dnes, freq='D')

# Výpis typů a hodnot pro dnešní datum a čas a pro dnešní periodu
print('Dnes:', type(dnes), dnes)
print('Dnes perioda:', type(dnes_perioda), dnes_perioda)

# Získání čísla dne v roce z dnešní periody
den_roku = dnes_perioda.dayofyear

# Výpis typu pro proměnnou dnes a čísla dne v roce
print('Den roku:', type(dnes), den_roku)


Dnes: <class 'datetime.datetime'> 2024-03-24 16:25:14.636250
Dnes perioda: <class 'pandas._libs.tslibs.period.Period'> 2024-03-24
Den roku: <class 'datetime.datetime'> 84


### Metoda DateOffset()

- Jedná se o metodu definující posun v čase.
- Představuje posun v čase, například `pd.to_datetime('today') + pd.DateOffset(days=1)` představuje zítřek.
- Zatímco `timedelta` je vhodnější pro posuny přesné časové intervaly, `DateOffset` je vhodnější pro posuny v kalendářních jednotkách.
- Možné hodnoty posunu jsou:
    - `years` - roky
    - `months` - měsíce
    - `weeks` - týdny
    - `days` - dny
    - `hours` - hodiny
    - `minutes` - minuty
    - `seconds` - sekundy
    - `microseconds` - mikrosekundy
    - `nanoseconds` - nanosekundy
- Kompletní přehled aliasů zde: [https://pandas.pydata.org/docs/reference/api/pandas.DateOffset.html#pandas.DateOffset](https://pandas.pydata.org/docs/reference/api/pandas.DateOffset.html#pandas.DateOffset).


In [11]:
import pandas as pd  # Import knihovny pandas pro práci s daty
import datetime  # Import modulu datetime pro práci s datem a časem

# Získání aktuálního data a času
dnes = datetime.datetime.now()
# Výpočet data začátku týdne (pondělí) pomocí pandas offsetu
zacatek_tydne = dnes - pd.offsets.Week(weekday=0)

# Výpis data začátku týdne
print('Zacatek tydne:', zacatek_tydne)


Zacatek tydne: 2024-03-18 16:25:14.643495


In [12]:
import pandas as pd  # Import knihovny pandas pro práci s datovými rámci

# Načtení datového souboru CSV z URL s určením delimiteru ';'
df = pd.read_csv('https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv', delimiter=';')

# Výpis datových typů sloupců v DataFrame
print(f'Datové typy sloupců: {df.dtypes}')

# Ponechání pouze sloupce DATE a konverze na DataFrame
df = df['DATE'].to_frame()
# Převod řetězců na datový typ datetime
df['DATUM'] = pd.to_datetime(df['DATE'])
# Vytvoření timedelta pro posun o jeden den
df['POSUN DEN'] = pd.to_timedelta('1D')
# Výpočet data o jeden den dříve
df['DATUM-1'] = df['DATUM'] - df['POSUN DEN']
# Vytvoření offsetu pro posun na začátek týdne
df['POSUN NA ZACATEK TYDNE'] = pd.offsets.Week(weekday=0)
# Vytvoření offsetu pro posun o jeden rok dozadu
df['POSUN O 1 ROK'] = pd.offsets.DateOffset(years=-1)
# Výpočet data odpovídajícího začátku týdne
df['ZACATEK TYDNE'] = df['DATUM'].apply(lambda x: x - pd.offsets.Week(weekday=0))
# Výpočet data o jeden rok zpět
df['DATUM-1 ROK'] = df['DATUM'].apply(lambda d: d - pd.offsets.DateOffset(years=1))

# Zobrazení prvních pěti řádků upraveného DataFrame
df.head()


Datové typy sloupců: STATION     object
NAME        object
DATE        object
PRCP       float64
SNWD       float64
TAVG       float64
TMAX       float64
TMIN       float64
dtype: object


DATE      DATUM POSUN DEN    DATUM-1 POSUN NA ZACATEK TYDNE  \
0  2006-12-01 2006-12-01    1 days 2006-11-30      <Week: weekday=0>   
1  2006-12-02 2006-12-02    1 days 2006-12-01      <Week: weekday=0>   
2  2006-12-03 2006-12-03    1 days 2006-12-02      <Week: weekday=0>   
3  2006-12-04 2006-12-04    1 days 2006-12-03      <Week: weekday=0>   
4  2006-12-05 2006-12-05    1 days 2006-12-04      <Week: weekday=0>   

            POSUN O 1 ROK ZACATEK TYDNE DATUM-1 ROK  
0  <DateOffset: years=-1>    2006-11-27  2005-12-01  
1  <DateOffset: years=-1>    2006-11-27  2005-12-02  
2  <DateOffset: years=-1>    2006-11-27  2005-12-03  
3  <DateOffset: years=-1>    2006-11-27  2005-12-04  
4  <DateOffset: years=-1>    2006-12-04  2005-12-05

### Cvičení1

1. Do proměnné df nahrajte data z github se srážkami.
2. Převeďte sloupec [DATE] na typ datetime64[ns].
3. Dopočítejte sloupec [ZACATEK TYDNE], ve kterém bude dopočteno datum začátku týdne hodnoty ze sloupce [DATE].
4. Dopočítejte sloupec [PREDCHOZI TYDEN], ve kterém bude dopočteno datum začátku týdne hodnoty ze sloupce [DATE] o týden zpět.


### Cvičení 2
1. Do proměnné df nahrajte data z github se srážkami.
2. Převeďte sloupec [DATE] na typ datetime64[ns].
3. Do proměnné df_datumy vytvořte date_range od minimální hodnoty sloupce [DATE] do maximální hodnoty sloupce [DATE].
4. Do proměnné df_agregovano agregujte df dle sloupce [DATE], kdy nás zajímá počet (count) hodnot ve sloupci [NAME].
5. Spojte zleva dataframe df_datumy a df_agregovano dle sloupce.
6. Určete, pro které [DATE] nemáme žádné hodnoty (tj. žádná stanice nevykázala informace o povětrnostních podmínkách).

